## 2. Import Dependencies

In [ ]:
import os
from deft_toolkit import *

## 3. Generate Failure Analyses

In [ ]:
model_list = [
    # "deepseek-v3.1",
    # "grok-4-0709",
    # "claude-opus-4-1-20250805",
    # "qwen3-max-preview",
    "gemini-2.5-pro"
]

for model_name in model_list:
    simplified_name = model_name.split('-')[0]
    generate_analyses(
        model=model_name,
        data=f"data/input/records.jsonl",
        out_file=f"data/input/records_with_{simplified_name}_analysis.jsonl",
        prompt_en_file="prompt/analyses_generation_en.txt",
        prompt_zh_file="prompt/analyses_generation_zh.txt",
        max_workers=15,
    )

## 4. Generate Categories (Modes)

In [ ]:
model_mapping = {
    "deepseek": "deepseek-v3.1",
    "grok": "grok-4",
    "claude": "claude-opus-4-1-20250805",
    "qwen3": "qwen3-max-preview",
    "gemini": "gemini-2.5-pro"
}

folder = "data/input"

for filename in os.listdir(folder):
    if filename.endswith("_analysis.jsonl"):
        simplified_name = filename.replace("records_with_", "").replace("_analysis.jsonl", "")
        model_name = model_mapping.get(simplified_name)
        data_file = os.path.join(folder, filename)

        generate_modes(
            model=model_name,
            data=data_file,
            out_file=f"data/output/files/generation_{simplified_name}.jsonl",
            mode_file=f"data/output/modes/modes_{simplified_name}.md",
            prompt_file="prompt/modes_generation.txt",
            seed_file="prompt/seeds.md",
        )

## 5. Refine Categories (Modes)

In [ ]:
refine_modes(
    model="gemini-2.5-pro",
    prompt_file="prompt/refinement.txt",
    mode_file="data/output/modes/modes_grok.md",
    out_file="data/output/modes/refinement.md",
    merge_threshold=0.6,
    remove_threshold=0.01,
)

## 6. Assign Categories (Modes) to Every Record

In [ ]:
assign_modes(
    model="gemini-2.5-pro",
    data="data/input/records.jsonl",
    out_file="data/output/files/records_annotated.jsonl",
    prompt_file="prompt/assignment.txt",
    mode_file="data/output/modes/final_DRAST.md",
    max_workers=15,
)

## 7. Compute Taxonomy Positive Metric (S)

In [ ]:
compute_metrics(
    data="data/output/files/records_annotated.jsonl",
    mode_file="data/output/modes/final_DRAST.md",
    output_col="responses"
)